## TPS Prediction with DNN and KerasTuner
In this notebook, I will build a DNN model to solve TPS problem with the help of KerasTuner. Since this dataset is so large, I will use a small dataset to do HyperParameter Tuning.

In [ ]:
import numpy as np
import pandas as pd 
from tensorflow import keras
from sklearn.model_selection import train_test_split
import keras_tuner as kt
import tensorflow as tf

In [ ]:
batch_size = 128

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-oct-2021/train.csv")

In [ ]:
train_set, val_set = train_test_split(train, test_size=0.15, random_state=42)

In [ ]:
portion = 0.01
train_subset = train_set.iloc[np.random.choice(train_set.shape[0], int(train_set.shape[0] * portion))]
val_subset = val_set.iloc[np.random.choice(val_set.shape[0], int(val_set.shape[0] * portion))]

In [ ]:
train_set.to_csv("train.csv", index=False)
val_set.to_csv("val.csv", index=False)
train_subset.to_csv("train_subset.csv", index=False)
val_subset.to_csv("val_subset.csv", index=False)

In [ ]:
del train
del train_set
del val_set
del train_subset
del val_subset

In [ ]:
train_ds = tf.data.experimental.make_csv_dataset("train.csv", batch_size=batch_size, label_name="target").shuffle(512).cache().prefetch(1)
val_ds = tf.data.experimental.make_csv_dataset("val.csv", batch_size=batch_size, label_name="target").cache().prefetch(1)
train_sub_ds = tf.data.experimental.make_csv_dataset("train_subset.csv", batch_size=batch_size, label_name="target").cache().prefetch(1)
val_sub_ds = tf.data.experimental.make_csv_dataset("val_subset.csv", batch_size=batch_size, label_name="target").cache().prefetch(1)
sample_submission = pd.read_csv("/kaggle/input/tabular-playground-series-oct-2021/sample_submission.csv")

In [ ]:
test_ds = tf.data.experimental.make_csv_dataset("/kaggle/input/tabular-playground-series-oct-2021/test.csv", batch_size=1000, shuffle=False)

In [ ]:
categorical_columns = ['f22', 'f43', 'f242', 'f243', 'f244', 'f245', 'f246', 'f247', 'f248', 'f249', 'f250', 'f251', 'f252', 'f253', 'f254', 'f255', 'f256', 'f257', 'f258', 'f259', 'f260', 'f261', 'f262', 'f263', 'f264', 'f265', 'f266', 'f267', 'f268', 'f269', 'f270', 'f271', 'f272', 'f273', 'f274', 'f275', 'f276', 'f277', 'f278', 'f279', 'f280', 'f281', 'f282', 'f283', 'f284']
numerical_columns = ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127', 'f128', 'f129', 'f130', 'f131', 'f132', 'f133', 'f134', 'f135', 'f136', 'f137', 'f138', 'f139', 'f140', 'f141', 'f142', 'f143', 'f144', 'f145', 'f146', 'f147', 'f148', 'f149', 'f150', 'f151', 'f152', 'f153', 'f154', 'f155', 'f156', 'f157', 'f158', 'f159', 'f160', 'f161', 'f162', 'f163', 'f164', 'f165', 'f166', 'f167', 'f168', 'f169', 'f170', 'f171', 'f172', 'f173', 'f174', 'f175', 'f176', 'f177', 'f178', 'f179', 'f180', 'f181', 'f182', 'f183', 'f184', 'f185', 'f186', 'f187', 'f188', 'f189', 'f190', 'f191', 'f192', 'f193', 'f194', 'f195', 'f196', 'f197', 'f198', 'f199', 'f200', 'f201', 'f202', 'f203', 'f204', 'f205', 'f206', 'f207', 'f208', 'f209', 'f210', 'f211', 'f212', 'f213', 'f214', 'f215', 'f216', 'f217', 'f218', 'f219', 'f220', 'f221', 'f222', 'f223', 'f224', 'f225', 'f226', 'f227', 'f228', 'f229', 'f230', 'f231', 'f232', 'f233', 'f234', 'f235', 'f236', 'f237', 'f238', 'f239', 'f240', 'f241']

In [ ]:
numeric_features = [tf.feature_column.numeric_column(item, dtype=tf.float32) for item in numerical_columns]
categorical_features = [tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_vocabulary_list(key, [0, 1])) for key in categorical_columns]

In [ ]:
features = numeric_features + categorical_features

In [ ]:
inputs = dict()
for feature in numeric_features:
    inputs[feature.key] = tf.keras.Input(name=feature.key, shape=())
for feature in categorical_features:
    inputs[feature.categorical_column.key] = tf.keras.Input(name=feature.categorical_column.key, shape=(), dtype="int32")

## Model Development

In [ ]:
def build_model(hp):
    x = tf.keras.layers.DenseFeatures(features)(inputs)
    for depth in range(hp.Choice('depth', [3, 4, 5, 6, 7, 8, 9, 10])):
        x = keras.layers.Dense(
            hp.Choice('width', [8, 16, 32, 64, 128, 256]), 
            activation='relu'
        )(x)
    output = keras.layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs=list(inputs.values()), outputs=output)
    adam = keras.optimizers.Adam(learning_rate=hp.Float("learing_rate", 1e-5, 5e-3))
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=["accuracy", keras.metrics.AUC()])
    return model

In [ ]:
train_length = pd.read_csv("train_subset.csv").shape[0]
val_length = pd.read_csv("val_subset.csv").shape[0]

In [ ]:
train_steps = train_length // batch_size
val_steps = val_length // batch_size

In [ ]:
tuner = kt.RandomSearch(
    build_model,
    objective=kt.Objective("val_auc", direction="max"),
    max_trials=50)
tuner.search(train_sub_ds.take(train_steps), epochs=3, validation_data=val_sub_ds.take(val_steps))
best_model = tuner.get_best_models()[0]
keras.utils.plot_model(best_model, show_shapes=True)

In [ ]:
best_hp = tuner.get_best_hyperparameters()[0]

Here is best parameters:

In [ ]:
for param in ["width", "depth", "learing_rate"]:
    print("%s:"%(param), best_hp.get(param))

In [ ]:
keras.backend.clear_session()

In [ ]:
model = tuner.hypermodel.build(best_hp)

In [ ]:
model_checkpoint_path = "model.h5"
early_stopping = keras.callbacks.EarlyStopping(patience=5)
model_checkpoint = keras.callbacks.ModelCheckpoint(model_checkpoint_path, save_best_only=True)
reduce_lr =  keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.00001)

In [ ]:
train_steps = int(1000000 // batch_size * 0.85)
val_steps = int(1000000 // batch_size * 0.15)

In [ ]:
history = model.fit(train_ds.take(train_steps), epochs=10, validation_data=val_ds.take(val_steps), callbacks=[early_stopping, model_checkpoint, reduce_lr])

In [ ]:
del train_ds
del val_ds

In [ ]:
pd.DataFrame(history.history).plot()

## Submission

In [ ]:
test_steps = 500000 // 1000
y_pred = model.predict(test_ds.take(test_steps)).reshape(-1)
print(y_pred.shape)
sample_submission["target"] = y_pred
sample_submission.to_csv("submission.csv", index=False)

## Conclusion
Even if I choose only 1% of data for hyper parameter tuning, KerasTuner can still find a good architecture that gets a good result.